# Logistic regession

## Các công thức

- sigmoid: $$σ(z) = \frac {1}{1 - e^{-z}}$$

- loss: $$J(W) = -\frac{1}{N} ∑_{i=1}^N (y_i \log z_i + (1 - y_i) \log(1-z_i)$$

- Gradient Descent cho logistic sigmoid regression: $$w ← w - λ(σ(w^T x_i) - y_i)x_i$$


In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dữ liệu 
Ở đây tôi dùng dữ liệu Breast cancer (sklearn) với:

- Class: 2
- number of sample: 569
- Dimension: 30

Chia thành hai tập train và test theo tỉ lệ 80% train và 20% test.

In [2]:
data = datasets.load_breast_cancer()

X, y = data['data'], data['target']

xtr, xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=101)
xtr.shape, xte.shape, ytr.shape, yte.shape

((455, 30), (114, 30), (455,), (114,))

In [3]:
def _sigmoid_function(x):
  if x >= 0:
    z = np.exp(-x)
    return 1 / (1 + z)
  else:
    z = np.exp(x)
    return z / (1 + z)


def sigmoid(x):
  return np.array([_sigmoid_function(value) for value in x])


def compute_loss(y_true, y_pred):
  # binary cross entropy
  # plus 1e-9 to ignore case y_pred equal zero
  y_zero_loss = y_true * np.log(y_pred + 1e-9)
  y_one_loss = (1-y_true) * np.log(1 - y_pred + 1e-9)
  return -np.mean(y_zero_loss + y_one_loss)


def compute_gradients(x, y_true, y_pred):
  difference =  y_pred - y_true
  gradient_b = np.mean(difference)
  gradients_w = np.matmul(x.transpose(), difference)
  gradients_w = np.array([np.mean(grad) for grad in gradients_w])
  return gradients_w, gradient_b


def predict(x, weights, bias):
  x_dot_weights = np.matmul(x, weights.transpose()) + bias
  probabilities = sigmoid(x_dot_weights)

  return [1 if p > 0.5 else 0 for p in probabilities]


def gradient_descent(X, y, learning_rate, epochs):
  weights = np.zeros(X.shape[1])
  bias = 0
  m = X.shape[0]
  train_accuracies = []
  losses = []

  for i in range(epochs):
    # compute f(X)
    x_dot_weights = np.matmul(weights, X.transpose()) + bias

    pred = sigmoid(x_dot_weights)

    loss = compute_loss(y, pred)
    
    # compute gradients
    error_w, error_b = compute_gradients(X, y, pred)
    
    # check for stop terminate
    if np.linalg.norm(error_w) / m < 1e-5 or np.linalg.norm(error_b) / m < 1e-5 or loss < 1e-5:
      break

    # update parameters
    weights = weights - learning_rate * error_w
    bias = bias - learning_rate * error_b

  return weights, bias, i

In [4]:
weights, bias, iter = gradient_descent(xtr, ytr, 0.01, 1000)

pred = predict(xte, weights, bias)

# Test accuracy on test dataset
accuracy = accuracy_score(yte, pred)
print(f"Train finished with accuracy {accuracy:.3f}, stop at iter={iter + 1}")

Train finished with accuracy 0.921, stop at iter=1000
